<a href="https://colab.research.google.com/github/kokutoubanira/NLP_Project_sample/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google Colaboratoryでivedoorニュース9カテゴリを分類する自然言語処理の実装について解説します。

※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

本実装の流れは、

1. livedoorニュースをダウンロードして、tsvファイルに変換
2. tsvファイルをPyTorchのtorchtextのDataLoaderに変換
3. クラス分類用のモデルを用意する
4. パラメータの設定
5. 学習の実施
6. テストデータでの性能を確認

1. livedoorニュースをダウンロードして、tsvファイルに変換
ここでは、livedoorニュースをダウンロードし、

本文[tab]クラスラベル
本文[tab]クラスラベル
本文[tab]クラスラベル

の構成のtsvファイルへと変換していきます。

まず、LiveDoorニュースをダウンロードします（乱数のシード固定は掲載を省略しています）。

### Janomeのインストール方法

コンソールにて、

- source activate pytorch_p36
- pip install janome

In [ ]:
!pip install janome

In [ ]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

ダウンロードした圧縮ファイルを解凍しカテゴリーの数と内容を確認します。

In [ ]:
# ファイルを解凍し、カテゴリー数と内容を確認
import tarfile
import os

# 解凍
tar = tarfile.open("ldcc-20140209.tar.gz", "r:gz")
tar.extractall("./data/livedoor/")
tar.close()

# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("./data/livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)

カテゴリーではない、ファイルなどもあるので、それを無視します。

ひとつ、ファイルの中身を確認してみましょう。

In [ ]:
# ファイルの中身を確認してみる
file_name = "./data/livedoor/text/movie-enter/movie-enter-6255260.txt"

with open(file_name) as text_file:
    text = text_file.readlines()
    print("0：", text[0])  # URL情報
    print("1：", text[1])  # タイムスタンプ
    print("2：", text[2])  # タイトル
    print("3：", text[3])  # 本文

    # 今回は4要素目には本文は伸びていないが、4要素目以降に本文がある場合もある

今回は、4要素目に本文が入っていませんが、4要素目以降に本文が入っている場合もあります。

この各ファイルから、タイトルは除いて、本文だけを抽出したtsvファイルを作成したいです。

タイトルの除くのは、タイトルは文章内容の要約であり、さすがにクラス分類のための情報量が多すぎるからです。

本文を取得する前処理関数を定義します。ここでは改行や全角スペースも削除しています。

In [ ]:
# 本文を取得する前処理関数を定義


def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text

この定義した前処理関数を利用して、全ファイルを変換します。
livedoorニュースの9カテゴリについて、各カテゴリーごとに処理を実施します。

In [ ]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)

リストをpandasのDataFrameに変換します。サイズを確認すると、7,376の文章があることが確認できます。

In [ ]:
# pandasのDataFrameにする
import pandas as pd

df = pd.DataFrame({'text': list_text, 'label': list_label})

# 大きさを確認しておく（7,376文章が存在）
print(df.shape)

df.head()

続いて、カテゴリー名を数値に変換する辞書を作成します。
そして、その辞書で数値に置き換えたDataFrameを用意します。

In [ ]:
# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

print(dic_id2cat)
print(dic_cat2id)

# DataFrameにカテゴリーindexの列を作成
df["label_index"] = df["label"].map(dic_cat2id)
df.head()

# label列を消去し、text, indexの順番にする
df = df.loc[:, ["text", "label_index"]]
df.head()

データがシャッフルされておらず、カテゴリーごとに固まっているので、シャッフルします。

In [ ]:
# 順番をシャッフルする
df = df.sample(frac=1, random_state=123).reset_index(drop=True)
df.head()

シャッフルされたデータの前2割をテストデータ、残りの8割は訓練&検証データとします。

結果、テストデータが1,475件、訓練&検証データが5,901件となります。

これをtest.tsv、train_eval.tsvとして、それぞれ保存します。

In [ ]:
# tsvファイルで保存する

# 全体の2割の文章数
len_0_2 = len(df) // 5

# 前から2割をテストデータとする
df[:len_0_2].to_csv("./test.tsv", sep='\t', index=False, header=None)
print(df[:len_0_2].shape)

# 前2割からを訓練&検証データとする
df[len_0_2:].to_csv("./train_eval.tsv", sep='\t', index=False, header=None)
print(df[len_0_2:].shape)

以上でlivedoorニュースのデータをtsvファイルに変換することができました。

なお、tsvファイルをGoogle Colaboratoryからダウンロードしたい場合は以下のfile.downloadのコメントを外して実行します。

In [ ]:
# tsvファイルをダウンロードしたい場合
from google.colab import files

# ダウンロードする場合はコメントを外す
# 少し時間がかかる（4MB）
# files.download("./test.tsv")


# ダウンロードする場合はコメントを外す
# 少し時間がかかる（18MB）
# files.download("./train_eval.tsv")

2. tsvファイルをPyTorchのtorchtextのDataLoaderに変換

続いて、作成したtsvファイルを、PyTorchで扱えるDataLoaderに変換します。
torchtextを使用します。
形態素解析のMeCabをインストールします。

In [ ]:
# MeCabとtransformersの用意
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install transformers==2.9.0

In [ ]:
from janome.tokenizer import Tokenizer

j_t = Tokenizer()

text = '機械学習が好きです。'

for token in j_t.tokenize(text):
    print(token)


In [ ]:
# 単語分割する関数を定義


def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]


text = '機械学習が好きです。'
print(tokenizer_janome(text))


MeCab

公式サイト

http://taku910.github.io/mecab/

In [ ]:
!pip install mecab-python3
!sudo apt install libmecab-dev
!sudo apt install mecab-ipadic-utf8


モデル（分類タスク用）の実装
本ファイルでは、クラス分類のモデルを実装します。
※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

In [ ]:
import random
import time
import numpy as np
from tqdm import tqdm
import torch 
from torch import nn
import torch.optim as optim
import torchtext

In [ ]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

形態素解析の準備

形態素解析はpipで簡単にインストールできるmecabのラッパーであるfugashiを使います。
下記のように辞書も一緒にpipでインストールできます。

In [ ]:
!pip install fugashi
!pip install unidic-lite

形態素解析をする関数は以下のように特に前処理などは施さないシンプルなものにしました。
fugashiは以下のように、mecabと同様の使い方ができます。

In [ ]:
import fugashi

tagger = fugashi.Tagger("-Owakati")
def make_wakati(text):
    text = tagger.parse(text)
    wakati = text.split(" ")
    wakati = list(filter(("").__ne__, wakati))
    return wakati

# 形態素解析テスト
text = df.sample(n=1)['text'].item()
print(make_wakati(text)[:30])

livedoorニュースコーパスを学習データ、検証データ、テストデータの3つに分割しています。
CNNの実装を確かめることがメインなので、単語ベクトルも今回はとりあえず学習データからvocabularyを生成して、ランダムなベクトルを扱うことにします。また、文章の最大長を指定するmax_lengthなどは特に指定していません。

In [ ]:
livedoor_df = df.copy()

In [ ]:
from sklearn.model_selection import train_test_split
from torchtext.legacy import data

# カテゴリーをidに変換します。
categories = livedoor_df['label_index'].unique().tolist()
livedoor_df['category_id'] = livedoor_df['label_index'].map(lambda x: categories.index(x))

# 元データを学習、検証、テストの３つに分割します。
train_val_df, test_df = train_test_split(livedoor_df[['text', 'label_index']], train_size=0.8)
train_df, val_df = train_test_split(train_val_df, train_size=0.75)

print('train size', train_df.shape)
print('validation size', val_df.shape)
print('test size', test_df.shape)
# train size (4425, 2)
# validation size (1475, 2)
# test size (1476, 2)

# torchtext用にtsvファイルで保存します。
train_df.to_csv('train.tsv', sep='\t', index=False, header=None)
val_df.to_csv('val.tsv', sep='\t', index=False, header=None)
test_df.to_csv('test.tsv', sep='\t', index=False, header=None)

TEXT = data.Field(sequential=True, tokenize=make_wakati, lower=False, batch_first=True, pad_token='<pad>')
LABEL = data.Field(sequential=False, use_vocab=False)

train_data, val_data, test_data = data.TabularDataset.splits(
    path="./", train='train.tsv', validation='val.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])

# vocabulary生成
# 学習データだけでvocabを作成します。
TEXT.build_vocab(train_data, min_freq=1)

BATCH_SIZE = 64
train_loader = data.Iterator(train_data, batch_size=BATCH_SIZE, train=True)
val_loader = data.Iterator(val_data, batch_size=BATCH_SIZE, train=False, sort=False)
test_loader = data.Iterator(test_data, batch_size=BATCH_SIZE, train=False, sort=False)

CNNによるモデルの定義

自然言語処理におけるCNNの実装解説
自然言語処理では文章を行列（単語ベクトルの集まり）として扱うことが多いですが、その行列を（チャネル1の）画像とみなせば、自然言語に対してCNNを適用することができます。

まずは文章の行列を用意します。要素がランダムなミニバッチサイズ2の7××5の行列を用意しています。
自然言語処理で考えると、長さが7, 単語のベクトル次元数が5の文章を2つ用意した、ということになります。

In [ ]:
import torch
import torch.nn as nn

mat = torch.rand(2, 7, 5)
print(mat.size())
#torch.Size([2, 7, 5])
print(mat)

この文章を畳み込みます。畳み込みフィルターのサイズは図の通り、4×5とし、ストライドは1とします。このフィルターを2枚畳み込みたいので、アウトプットのチャネルは2を指定すればOK。
自然言語処理でnn.LSTMなどを扱うとき、インプットの形式は（batch_first=Trueを指定した場合）ミニバッチサイズ×文章の長さ×単語ベクトル次元数のテンソルを扱いますが、nn.Conv2dのインプットの形式はミニバッチサイズ×チャネル数×高さ×幅である必要があります。なので、下記のようにmat.unsqueeze(1)をしてミニバッチサイズの次にチャネル1の次元を追加しています。

In [ ]:
# 第１引数はインプットのチャネル（今回は1）を指定
# 自然言語処理で畳み込む場合、異なる単語分散表現（word2vecとfasttextみたいな）などを使って、
# 複数チャネルとみなす方法もあるようです。
# 第２引数はアウトプットのチャネル数で、今回は同じフィルターを2枚畳み込みたいので、2を指定
# カーネルサイズは高さ×幅を指定しており、幅は図で説明した通り、単語ベクトルの次元数5を指定
conv = nn.Conv2d(1, 2, kernel_size=(4, 5))

# チャネル数1を挿入
mat = mat.unsqueeze(1)
print(mat.size())
# torch.Size([2, 1, 7, 5]) 
# ↑ミニバッチサイズ×チャネル数×文章の長さ×単語ベクトル次元数

# 畳み込む
feature = conv(mat)
print(feature.size())
# torch.Size([2, 2, 4, 1])
# ↑ミニバッチサイズ×特徴マップの数×（特徴マップの形式4×1）
print(feature)

In [ ]:
import torch.nn.functional as F
feature = F.relu(feature)
print(feature.size())
# ↑ミニバッチサイズ×特徴マップの数×（特徴マップの形式4×1）
print(feature)

1-max poolingを行います。ここですることは要は各特徴マップの最大要素を抽出することになります。つまりnn.MaxPool2dを使って、

In [ ]:
# nn.MaxPool1dでも良いですが、そのときは上のfeatureに対して
# feauture.unsqueeze(-1)をして最後の次元の1を除去しましょう。
pool = nn.MaxPool2d(kernel_size=(4, 1))
print(pool(feature))

としたくなりますが、上のpoolingのカーネルサイズ(上の4のところ)はインプットとなる特徴マップのサイズに依存するんですよね。特徴マップのサイズは元の文章の長さ（最初の行列の行方向）と畳み込みフィルターのサイズに依存するので、上のようにnn.MaxPool2dでpoolingするレイヤーのインスタンスを宣言しちゃうと、poolingのカーネルサイズを可変にできないので、F.max_pool2dを使って以下のようにpoolingする際、インプットとなる特徴マップのサイズを指定するようにします。

In [ ]:
# feature.size()[2]で特徴マップの高さを取得しています。
feature = F.max_pool2d(feature, kernel_size=(feature.size()[2], 1))
print(feature.size())
# torch.Size([2, 2, 1, 1])
print(feature)
# viewを使って次元数を整頓します。
feature = feature.view(-1, 2)
print(feature.size())
# torch.Size([2, 2])
print(feature)

あとは同様のことを異なる畳み込みフィルターにも適用して、最後に要素を結合して全結合層にぶち込めばOKですね。

ネットワークの定義

In [ ]:
class Net(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Net, self).__init__()
        # 単語分散表現はランダムベクトルを使う
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=TEXT.vocab.stoi['<pad>'])
        # 図の黄色い畳み込みフィルター
        self.conv1 = nn.Conv2d(1, 2, kernel_size=(2, embedding_dim))
        # 図の緑色の畳み込みフィルター
        self.conv2 = nn.Conv2d(1, 2, kernel_size=(3, embedding_dim))
        # 図の赤色の畳み込みフィルター
        self.conv3 = nn.Conv2d(1, 2, kernel_size=(4, embedding_dim))

        # 3つ畳み込みの処理でそれぞれ2次元のベクトルが生成されるので、それらを全て結合して6次元のベクトルとなります。
        # livedoorのカテゴリは9つなので、アウトプットサイズは9を指定
        self.linear = nn.Linear(6, 9)

    def forward(self, input_ids):
        # ①文章の行列を取得
        out = self.embeddings(input_ids)
        # チャネル数1を挿入
        out = out.unsqueeze(1)

        # ②畳み込んでreluに通す
        out1 = F.relu(self.conv1(out))
        out2 = F.relu(self.conv2(out))
        out3 = F.relu(self.conv3(out))

        # ③poolingして、各特徴マップの最大要素を取得
        out1 = F.max_pool2d(out1, kernel_size=(out1.size()[2], 1))
        out2 = F.max_pool2d(out2, kernel_size=(out2.size()[2], 1))
        out3 = F.max_pool2d(out3, kernel_size=(out3.size()[2], 1))

        # ④viewして次元を整えてあげる
        out1 = out1.view(-1, 2)
        out2 = out2.view(-1, 2)
        out3 = out3.view(-1, 2)

        # ⑤全部結合して1本のベクトルにする
        out = torch.cat([out1, out2, out3], dim=1)

        # ⑥全結合層で９つのカテゴリー分類できるように変換
        out = self.linear(out)

        return out

学習
あとはこのネットワークでちゃんと学習できるか確かめて精度を確認して終わりです。
学習部分は以下のように実装しました。
学習データ、検証データともに順調に損失は減っていきますが、検証データの損失が最後らへんで増えてしまいます。

In [ ]:
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm 
VOCAB_SIZE = len(TEXT.vocab.stoi)
EMBEDDING_DIM = 200

net = Net(VOCAB_SIZE, EMBEDDING_DIM)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# GPUの設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# ネットワークをGPUへ送る
net.to(device)
train_loss = []
val_loss = []
train_accuracy = []
val_accuracy = []

for epoch in tqdm(range(30)):

    # 学習
    _train_loss = 0.0
    _train_acc = 0.0
    net.train()
    for batch in tqdm(train_loader):
        inputs = batch.Text.to(device)
        y = batch.Label.to(device)
        optimizer.zero_grad()
        out = net(inputs)
        loss = loss_function(out, y)
        _, preds = torch.max(out, 1)
        loss.backward()
        optimizer.step()
        _train_loss += loss.item()
        _train_acc += torch.sum(preds == y).item()
    train_loss.append(_train_loss)
    train_epoch_acc = _train_acc / len(train_loader.dataset)
    train_accuracy.append(train_epoch_acc)

    # 検証
    _val_loss = 0.0
    _val_acc = 0.0
    net.eval()
    with torch.no_grad():
        for batch in val_loader:
            inputs = batch.Text.to(device)
            y = batch.Label.to(device)
            out = net(inputs)
            loss = loss_function(out, y)
            _, preds = torch.max(out, 1)
            _val_loss += loss.item()
            _val_acc += torch.sum(preds == y).item()
    val_loss.append(_val_loss)
    val_epoch_acc = _val_acc / len(val_loader.dataset)
    val_accuracy.append(val_epoch_acc)

    print("epoch", epoch,
          "\ttrain loss", round(_train_loss, 4), "\ttrain accuracy", round(train_epoch_acc, 4),
          "\tval loss", round(_val_loss, 4), "\tval accuracy", round(val_epoch_acc, 4))

精度確認
最後にテストデータによる精度（Fスコア）を確認しましょう。

In [ ]:
# 精度確認
from sklearn.metrics import classification_report

with torch.no_grad():
    test_loss = 0.0
    net.eval()
    prediction = []
    answer = []
    for batch in test_loader:
        input_ids = batch.Text.to(device)
        y = batch.Label.to(device)
        out = net(input_ids)
        _, preds = torch.max(out, 1)
        prediction += list(preds.cpu().numpy())
        answer += list(y.cpu().numpy())
print(classification_report(prediction, answer, target_names=categories))

おわりに
自然言語処理に対するCNNの適用例ということでCNNによる文章分類の実装を確認しました。
